Change the below to point to your Parquet file example if you have one

In [ ]:
example_parquet_file_location = "../data/20221109.parquet"

In [ ]:
import numpy as np
import pandas as pd

Use the below code to generate a Parquet file with random data similar to a series of trade prices

In [ ]:
number_of_rows = 100000
low_size, high_size = 10, 10000
low_price, high_price = 15000.0, 18000.0

In [ ]:
df = pd.DataFrame({
    'time': sorted(np.random.randint(0,1e13, size=number_of_rows)),
    'side': np.random.choice(['B', 'S'],number_of_rows),
    'size': np.random.randint(low_size, high_size, size=number_of_rows),
    'price': np.random.uniform(low_price, high_price, number_of_rows)
})

In [ ]:
example_parquet_file_location = "../data/random_trade_like_data.parquet"

Write the dataframe out to the Parquet file

This uses an existing Parquet interface

In [ ]:
df.to_parquet(example_parquet_file_location)

Add location of shared library to search path

In [ ]:
import sys
sys.path.append("../build")

The build places the shared library under 'lib'

In [ ]:
import lib.parquet_table as pqt

In [ ]:
pt = pqt.ParquetTable(example_parquet_file_location)

In [ ]:
pt.print_stats()

In [ ]:
pt.column(3)

In [ ]:
import datetime
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame({ 
    "time": pt.column(0),
    "price": pt.column(3),
    "size": pt.column(2)
})

In [ ]:
df.tail()

In [ ]:
def parse_time(x):
    x = datetime.datetime.fromtimestamp(x/1e6)
    return x.strftime('%b-%d %H:%M')
    
df['time_bin'] = df['time'].apply(parse_time)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
tf = df.groupby('time_bin').agg({
    'size': np.sum,
    'price': lambda x: x.iloc[-1]
}).reset_index()
tf.head()

In [ ]:
tf.plot(x='time_bin', y=['size'], figsize=(16, 8))

In [ ]:
tf.plot(x='time_bin', y=['price'], figsize=(16, 8))

In [ ]:
from fastparquet import ParquetFile
pf = ParquetFile("../data/20221109.parquet")

In [ ]:
df = pf.to_pandas()